1_Read dataset
2_Encoding
3_Normalization
4_Train_test_split
5_Train_model


In [ ]:
import pandas as pd

from Projects.Functions.Config_Loader.config_loader import *
from Projects.Functions.Features_Engineering.features_engineering import *
from Projects.Functions.Modelisation.modelisation import *
from Projects.Functions.Models_generation.models_generation import *
from Projects.Functions.Report.report import *
from Projects.Functions.Pipeline_Modelisation.pipeline_modelisation import *
from scipy.stats import iqr, shapiro,kstest

In [ ]:
config=ConfigLoader(r"C:\Users\tallar\Documents\PROJETS\EDA_and_ML\Projects\Configs\Config.json")

In [ ]:
dataset=config.get_dataset_path()
print(f"dataset_path : {dataset}")
cible=config.get_project_target()
print(f"cible : {cible}")
modele_type=config.get_model_type()
print(f"model_type : {modele_type}")
test_fraction=config.get_test_fraction()
print(f"test_fraction : {test_fraction}")
model_path=config.get_model_path()
print(f"model_path : {model_path}")
alpha = config.get_alpha()
print(f"alpha : {alpha}")
correlation_limit = config.get_corr_limit()
print(f"correlation_limit : {correlation_limit}")
cross_validation =config.get_cross_validation()
print(f"cross_validation : {cross_validation}")
file_name =config.get_dataset_filename()
print(f"file_name : {file_name}")
columns_to_delete = config.get_deleted_columns()
print(f"columns_to_delete : {columns_to_delete}")
features_path = config.get_features_json_path()
print(f"features_path : {features_path}")
figure_path = config.get_figure_path()
print(f"figure_path : {figure_path}")
model_api_path=config.get_model_api_path()
print(f"model_api_path : {model_api_path}")
k_fold = config.get_fold_count()
print(f"k_fold : {k_fold}")
pca_dim = config.get_pca_dim()
print(f"pca_dim : {pca_dim}")
pca_variance = config.get_pca_variance()
print(f"pca_variance : {pca_variance}")
pca=config.use_pca()

In [ ]:
project_name="House_price"
project_name = project_name + " version_"+ str(datetime.now().year) +"_"+ str(datetime.now().month) +"_"+ str(datetime.now().day) +"_"+datetime.now().strftime("%H%M%S")
report=Report(project_name)

In [ ]:
df= dataset_reading(dataset)
cible=cible
raw_features=df.drop(cible, axis=1)
raw_features= dataset_object_to_categorical(raw_features)
raw_features=dataset_na_fill(raw_features, cible)


raw_targets=df[cible]

In [ ]:
df = dataset_object_to_categorical(df)
df=dataset_na_fill(df,cible)

In [ ]:

features, target = encoding_categorical_features(df,cible)
print(features)

In [ ]:
features_train, features_test, target_train, target_test = split_dataset(features,target,test_fraction,42)

In [ ]:
features, features_train_scaled, features_test_scaled = scale_features(features_train,features_test)
if pca==True:
    features_train, features_test, n_composantes = PCA_dimension_reduction(features_train_scaled,features_test_scaled,pca_variance,pca_dim)

In [ ]:
modeles=generate_list_of_models(modele_type)

In [ ]:
pipeline_model, predictions, recall,accuracy,f1score,precision,mse, r2, tab_modele, dict_modeles =training_machine_learning_models(modele_type,modeles,False, features_train_scaled,features_test_scaled,target_train,target_test)

print(tab_modele)

In [ ]:
best_modele, pipeline_best_model, best = evaluation_of_trained_models(predictions , dict_modeles,pipeline_model, modele_type, target_test, recall,accuracy,f1score,precision,mse, r2, tab_modele, figure_path)

In [ ]:
pd.DataFrame(predictions).head()

In [ ]:
print(best)

In [ ]:
results = generate_results(tab_modele,modele_type)

In [ ]:
results.head()

In [ ]:
report.doc_add_table(results)

In [ ]:
print(config.get_report_path())

In [ ]:
pipeline = build_the_final_pipeline(pipeline_best_model,raw_features,raw_targets,cible,pca,pca_dim)

In [ ]:
saving_of_best_model_for_production(model_path,model_api_path,pipeline,best,project_name)

In [ ]:
generate_data_contract_for_frontEnd(raw_features,features_path)

In [ ]:
report.report_builder(config.get_report_path())